<a href="https://colab.research.google.com/github/mkielo3/mammalian_brains/blob/main/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Demonstration of Receptive Field and SOM probes for Dual Computational Systems in the Development

The notebook runs the computational probes in Figure 4 of: "Dual Computational Systems in the Development and Evolution of Mammalian Brains" by Imam et al.

By default, probes will result in a downsampled 5x5 SOM (rather than 25x25 show in the paper) and will complete in about 10 minutes. To run the full 25x25 SOM, change the fast=True flag to False. The full size SOM completes in 1-2 hours.

In [1]:
import sys
if 'google.colab' in sys.modules:
	%pip install -q numpy pandas scipy scikit-learn matplotlib seaborn pillow ipykernel torch torchvision torchaudio altair datasets numba numbasom==0.0.5 tensorboard pysal vl-convert-python soundfile

	!git clone https://github.com/mkielo3/mammalian_brains.git
	%cd mammalian_brains

In [2]:
import torch
import pandas as pd
from tqdm import tqdm
from numbasom.core import lattice_closest_vectors
from analysis.som import SOM
from utils import save_output_to_pickle
from models.olfaction.olfaction import Olfaction
from models.vision.vision import Vision
from models.audio.audio import Audio
from models.touch.touch import Touch
from models.memory.memory import Memory
from utils import save_som_plot, save_rf_plot
from config import Args

# Set all seeds
from numba import njit
import numpy as np

@njit
def set_numba_seed(value):
    np.random.seed(value)

SEED = 123
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(SEED)
import random
random.seed(SEED)
set_numba_seed(SEED)

args = Args()
fast = True
args.som_size = (5,5) if fast else (25, 25)
args.experiment_name = "main_results_fast" if fast else "main_results"
args.fast = fast
args.som_epochs = 2000 if fast else 100000
modality_list = [Olfaction(args), Vision(args), Audio(args), Touch(args), Memory(args)]

/Users/matthewkielo/miniforge3/envs/babylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
for modality in modality_list:
	print ("\n", modality.modality, pd.Timestamp.now())

	# 1. Train/Download Model
	modality.setup_model()
	modality.setup_som()

	# 2. Get activations for each patch
	patches = modality.get_patches()
	activation_list = []
	for p in tqdm(patches):
		p, static = modality.generate_static(p)
		activation = modality.calculate_activations(static)
		activation_list.append([p, activation])

	# 3. Fit SOM
	x_mat = torch.stack([x[1] for x in activation_list]).numpy()
	som = modality.initialize_som(SOM)
	lattice = som.train(x_mat, num_iterations=args.som_epochs, initialize=args.som_init, normalize=False, start_lrate=args.som_lr)

	# 4. Get coordinates for each BMU
	coordinate_list = [x[0] for x in activation_list]
	closest = lattice_closest_vectors(x_mat, lattice, additional_list=coordinate_list)

	# 5. Save
	output = {"closest": closest,
			"coord_map": coordinate_list,
			"x_range": (0, max([x[0][0] for x in activation_list])),
			"y_range": (0, max([x[0][1] for x in activation_list])),
			"lattice": lattice,
			"som": None,
			"samples": modality.sample_data,
			"modality": modality.modality,
			"args": args,
			"activations": activation_list}

	save_output_to_pickle(output, args.experiment_name)




 olfaction 2025-02-17 13:41:17.105899


100%|██████████| 4096/4096 [00:02<00:00, 1597.96it/s]


Initializing SOM with Random
Done Init
SOM training took: 1.315296 seconds.
Finding closest data points took: 0.090761 seconds.
Saved output to: output/main_results_fast/olfaction.pkl

 vision 2025-02-17 13:41:21.487038


Using cache found in /Users/matthewkielo/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/matthewkielo/miniforge3/envs/babylm/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/matthewkielo/miniforge3/envs/babylm/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 225/225 [00:06<00:00, 32.23it/s]


Initializing SOM with Random
Done Init
SOM training took: 1.614381 seconds.
Finding closest data points took: 0.144583 seconds.
Saved output to: output/main_results_fast/vision.pkl

 audio 2025-02-17 13:41:30.663110


...

Initializing SOM with Random
Done Init
SOM training took: 2.179545 seconds.
Finding closest data points took: 0.107330 seconds.
Saved output to: output/main_results_fast/audio.pkl

 touch 2025-02-17 13:42:35.072041
[TouchDataset] Refreshing tuples...
Loaded "ObjectClusterDataset" - split "test" with 16119 records...
[TouchDataset] Refreshing tuples...
Base LR = 1.000000e-03
tensor(0.5800)


100%|██████████| 548/548 [00:01<00:00, 351.76it/s]


Initializing SOM with Random
Done Init
SOM training took: 0.560841 seconds.
Finding closest data points took: 0.243159 seconds.
Saved output to: output/main_results_fast/touch.pkl

 memory 2025-02-17 13:42:40.751315


100%|██████████| 4455/4455 [00:00<00:00, 214235.55it/s]


Initializing SOM with Random
Done Init
SOM training took: 0.029393 seconds.
Finding closest data points took: 0.045698 seconds.
Saved output to: output/main_results_fast/memory.pkl


In [4]:
save_som_plot(args.experiment_name, modality_list, args)

alt.HConcatChart(...)

In [5]:
save_rf_plot(args.experiment_name, modality_list)

range(0, 5)


alt.HConcatChart(...)